# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Cuanto tarda en notarse en la generación eléctrica de fuentes de energía renovables los fenomenos metereológicos.
- Que relación hay entre precipitaciones y el aumento de la generación de energia eléctrica hidráulica.
- Existe realmente una relación entre la metereologia y la producción electrica.


Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Debido a la falta de datos a nivel diario de REE por provincia o comunidades autonomas, voy a centrar el analisis a nivel de sistema eléctrico (Peninsula, Baleares, Canarias ,Ceuta y Melilla).

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Requisitos para la ejecución del notebook:

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request



In [1]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

###  Lectura datos AEMET

Para la lectura de datos metereólogicos, vamos a utilizar la libreria aemet, de la que utilizaremos los metodos de las clases Aemet y Estacion para obtener los datos a nivel diario de cada estación meterologica para un rago de fechas, de las provincias que nos interesan para el estudio.

In [2]:
# Obtenemos el json de estaciones de mediciones de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)

In [4]:

# Definimos funciones que vamos a utilizar para leer los datos de AEMET
def estaciones_prov (prov,lista_estaciones):
    '''Dada una lista de provincias y un json de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    prov=list(map(str.upper, prov))
    for estacion in lista_estaciones:
        if estacion['provincia'] in prov:
            lista_id.append(estacion['indicativo'])
    return lista_id


def lectura_diaria_json(date_ini,date_end,estaciones):
    '''Dado un json de la clase Estacion de la libreria Aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    valores_error=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas por minuto.
            #Volvemos a intentar leer el dato que ha lanzado la excepcion.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;


def lectura_diaria_lista(date_ini,date_end,lista_estaciones):
    '''Dado una lista de id de estaciones de aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in lista_estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;

In [149]:
# Parametros para lectura de datos de AEMET

provincias=['Ceuta','Melilla','Illes Balears','Las Palmas','Sta. Cruz de Tenerife']

date_ini="2019-12-30T00:00:00UTC"
date_end="2019-12-30T23:59:59UTC"

id_estaciones=estaciones_prov(provincias,info_estaciones)

# Pasamos los datos ha dataframes para su procesado y limpieza
estaciones=pd.DataFrame(info_estaciones)
df_weather=pd.DataFrame(lectura_diaria_lista(date_ini,date_end,id_estaciones),dtype=str)

### Limpieza datos AEMET

In [150]:
# Creo una funcion que pase los indicadores a float y rellene los valores vacios por la temperatura media de las estaciones de esa provincia
def rellena_nulos_provincia(df,cols):
    # Defino un DataFrame vacio para acumular el resultado
    df_all=pd.DataFrame()
    
    # Hago un bucle para cada provincia del DataFrame de entrada
    for prov in df['provincia'].unique():
        df_prov=df[df['provincia']==prov].copy()
        
        #Para cada elemento de las columnas que nos interesan reemplazo , por ., paso a numerico y relleno los nulos por la media de la provincia
        for element in cols:
            df_prov[element]=df_prov[element].str.replace(',', '.')
            df_prov[element]=pd.to_numeric(df_prov[element],errors='coerce')
            df_prov[element].fillna(df_prov[element].mean(skipna=True),inplace=True)
        df_all=df_all.append(df_prov)
    return df_all

In [151]:
# Elimino las columnas que no me interesan
df_weather.drop(columns=['horaPresMax','horaPresMin','horaracha','dir','horatmin','horatmax'],inplace=True)

# Limpio de nulos la muestra
rellena_nulos_provincia(df_weather,df_weather.columns[5:])


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,velmedia,racha,sol,presMax,presMin
0,2019-12-30,5000C,CEUTA,CEUTA,87,15.000000,0.0,13.700000,16.400000,4.700000,11.100000,3.800000,1021.80,1018.700000
1,2019-12-30,6000A,MELILLA,MELILLA,52,14.700000,0.0,12.600000,16.800000,2.500000,6.400000,3.800000,1026.20,1023.300000
2,2019-12-30,B569X,CAPDEPERA,ILLES BALEARS,57,14.600000,0.0,11.900000,17.400000,1.400000,4.400000,6.400000,1030.25,1027.966667
3,2019-12-30,B013X,"ESCORCA, LLUC",ILLES BALEARS,490,7.400000,0.0,1.400000,13.500000,1.733333,5.288889,6.400000,1030.25,1027.966667
4,2019-12-30,B954,"IBIZA, AEROPUERTO",ILLES BALEARS,6,11.400000,0.0,6.600000,16.200000,1.400000,4.700000,1.600000,1031.80,1029.300000
5,2019-12-30,B893,"MENORCA, AEROPUERTO",ILLES BALEARS,91,9.800000,0.0,4.100000,15.600000,3.100000,6.100000,4.400000,1023.10,1021.000000
6,2019-12-30,B278,"PALMA DE MALLORCA, AEROPUERTO",ILLES BALEARS,8,10.300000,0.0,4.300000,16.300000,1.900000,5.800000,7.800000,1033.30,1030.900000
7,2019-12-30,B228,"PALMA, PUERTO",ILLES BALEARS,3,12.700000,0.0,8.700000,16.700000,0.600000,5.000000,8.600000,1032.30,1030.400000
8,2019-12-30,B346X,PORRERES,ILLES BALEARS,120,11.300000,0.0,5.700000,16.900000,1.100000,4.400000,6.400000,1030.25,1027.966667
9,2019-12-30,B434X,PORTOCOLOM,ILLES BALEARS,17,12.400000,0.0,7.400000,17.400000,2.200000,4.700000,7.700000,1031.10,1028.700000


# Análisis de la muestra

1 Evolución de temperaturas

2 Correlaciones

3 Evolución de precipitaciones

4 Datos estacionales

5 Localizar datos anomalos


## Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [152]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('REGION_REE',header=0,index_col='Region')

# Me quedo solo con los distintos sistemas electricos existentes
region_system=region_ree[region_ree['geo_limit']!='ccaa']

region_system

,geo_limit,geo_id
Region,,
peninsular,peninsular,8741
canarias,canarias,8742
baleares,baleares,8743
ceuta,ceuta,8744
melilla,melilla,8745


In [153]:
geo_limit=region_system[region_system['geo_id']==8745]['geo_limit']
geo_limit[0]

'melilla'

In [152]:
# Obtenemos los datos de REE a traves de su API.




parametros={'start_date':'2020-12-31T00:00',
            'end_date':'2021-01-20T00:00',
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':'canarias',
            'geo_ids':'8742'}

URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

ree_gen=requests.get(URL_GEN,params=parametros)
  
if ree_gen.raise_for_status()!=None:
    print('Error en lectura')


In [157]:

    df_ree=pd.DataFrame()
    for i in range(10):
        df=pd.json_normalize(ree_gen.json()['included'][i]['attributes'],meta=['title','type'],record_path=['values'])
        df_ree=df_ree.append(df)

    df_ree.reset_index(inplace=True,drop=True)


,value,percentage,datetime,title,type
0,9.902,0.000446,2020-12-31T00:00:00.000+01:00,Hidráulica,Renovable
1,5443.054,0.245243,2020-12-31T00:00:00.000+01:00,Motores diésel,No-Renovable
2,4260.363,0.216781,2021-01-01T00:00:00.000+01:00,Motores diésel,No-Renovable
3,5212.173,0.244999,2021-01-02T00:00:00.000+01:00,Motores diésel,No-Renovable
4,5229.706,0.253130,2021-01-03T00:00:00.000+01:00,Motores diésel,No-Renovable
...,...,...,...,...,...
163,20931.266,1.000000,2021-01-16T00:00:00.000+01:00,Generación total,Generación total
164,19821.599,1.000000,2021-01-17T00:00:00.000+01:00,Generación total,Generación total
165,21840.651,1.000000,2021-01-18T00:00:00.000+01:00,Generación total,Generación total
166,21879.485,1.000000,2021-01-19T00:00:00.000+01:00,Generación total,Generación total


In [178]:
def lectura_ree_electric_system(d_inicio,d_fin,geo_id):
    
    # meter esto en una funcion con su try-exception    
    
    geo_limit=region_system[region_system['geo_id']==geo_id]['geo_limit']
    
    parametros={'start_date':date_ini,
            'end_date':date_end,
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':geo_limit[0],
            'geo_ids':geo_id}
    
    URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

    ree_gen=requests.get(URL_GEN,params=parametros)
    
    df_ree=pd.DataFrame()
    for i in range(20):
        try:
            df=pd.json_normalize(ree_gen.json()['included'][i]['attributes'],meta=['title','type'],record_path=['values'])
            df['system']=geo_limit[0]
            df_ree=df_ree.append(df)
        except:
            pass #Cuando no hay datos para mas tecnologías
    df_ree.reset_index(inplace=True,drop=True)
    
    return df_ree

In [179]:
df_ree_system=pd.DataFrame()
for electric_system in region_system['geo_id']:
    df_ree_system=df_ree_system.append(lectura_ree_electric_system(date_ini,date_end,electric_system))
df_ree_system   

,value,percentage,datetime,title,type,system
0,162226.1190,0.275073,2019-12-30T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
1,6602.9850,0.280841,2019-12-31T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
2,6815.5810,0.011557,2019-12-30T00:00:00.000+01:00,Turbinación bombeo,No-Renovable,peninsular
3,28.5870,0.001216,2019-12-31T00:00:00.000+01:00,Turbinación bombeo,No-Renovable,peninsular
4,163461.2160,0.277167,2019-12-30T00:00:00.000+01:00,Nuclear,No-Renovable,peninsular
...,...,...,...,...,...,...
4,0.6605,0.031050,2019-12-31T00:00:00.000+01:00,Residuos no renovables,No-Renovable,melilla
5,13.2770,0.025352,2019-12-30T00:00:00.000+01:00,Residuos renovables,Renovable,melilla
6,0.6605,0.031050,2019-12-31T00:00:00.000+01:00,Residuos renovables,Renovable,melilla
7,523.7020,1.000000,2019-12-30T00:00:00.000+01:00,Generación total,Generación total,melilla
